In [1]:
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis import solver_demo_helper_functions as solver_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import numpy as np

def define_sample_problem_geom(ele_type, nx, ny, L, H):
    coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)
    return coords, connect

def define_sample_problem_info(ele_type, coords, connect, L, H):
    # Identify boundaries
    boundary_nodes, boundary_edges = pre.identify_rect_boundaries(coords, connect, ele_type, 0.0, L, 0.0, H)
    # 1. Fix left boundary: both u_x and u_y = 0.
    fixed_nodes = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)
    # Assign distributed load on the right boundary
    q = 10.0
    dload_info = pre.assign_uniform_load_rect(boundary_edges, "right", q, 0.0)
    # Assign material properties
    mu = 10
    kappa = 100
    material_props = np.array([mu, kappa])
    # Assign artificial displacement field
    displacement = np.zeros((coords.shape))
    for kk in range(0, coords.shape[0]):
        displacement[kk, 0] = coords[kk, 0] * 0.01
    return displacement, material_props, fixed_nodes, dload_info


In [2]:
ele_type = "D2_nn8_quad" # change this!
nx = 50 # change this to refine the mesh
ny = 50 # change this to refine the mesh
L = 10
H = 10
coords, connect = define_sample_problem_geom(ele_type, nx, ny, L, H)
displacement, material_props, fixed_nodes, dload_info = define_sample_problem_info(ele_type, coords, connect, L, H)

fname = "solver_mesh_1.png"
pre_demo.plot_mesh_2D(fname, ele_type, coords, connect)


In [3]:
# test how long assembley takes

num_runs = 5

avg_time_global_stiffness = solver_demo.time_assemble_global_stiffness(
        num_runs=num_runs,
        ele_type=ele_type,
        coords=coords.T,
        connect=connect.T,
        material_props=material_props,
        displacement=displacement.T
    )

avg_time_global_traction = solver_demo.time_assemble_global_traction(
        num_runs=num_runs,
        ele_type=ele_type,
        coords=coords.T,
        connect=connect.T,
        dload_info=dload_info
    )

avg_time_global_residual = solver_demo.time_assemble_global_residual(
        num_runs=num_runs,
        ele_type=ele_type,
        coords=coords.T,
        connect=connect.T,
        material_props=material_props,
        displacement=displacement.T
    )

print("avg time global stiffness:", avg_time_global_stiffness, "seconds")
print("avg time global traction:", avg_time_global_traction, "seconds")
print("avg time global residual:", avg_time_global_residual, "seconds")

avg time global stiffness: 1.113007933201152 seconds
avg time global traction: 0.0024896415998227894 seconds
avg time global residual: 0.9438154500006931 seconds


In [4]:
# test how long the matrix solve takes

K, R = solver_demo.prep_for_matrix_solve(ele_type, coords.T, connect.T, material_props, displacement.T, fixed_nodes, dload_info)
print(K.shape[0])
method = "dense"
num_runs = 5
avg_time_dense_solve = solver_demo.time_one_matrix_solve(K, R, method, num_runs)

print("average time dense matrix solve:", avg_time_dense_solve, "seconds")


15402
average time dense matrix solve: 30.236315433400158 seconds


In [5]:
# look at the stiffness matrix to see sparse-ness

fname = "solver_global_stiffness_1.png"
solver_demo.analyze_and_visualize_matrix(K, fname)

In [6]:
# try a sparse matrix assembly option

num_runs = 5

avg_time_global_stiffness_sparse = solver_demo.time_assemble_global_stiffness_sparse(
        num_runs=num_runs,
        ele_type=ele_type,
        coords=coords.T,
        connect=connect.T,
        material_props=material_props,
        displacement=displacement.T
    )

print("avg time global stiffness assembly sparse:", avg_time_global_stiffness_sparse, "seconds")

avg time global stiffness assembly sparse: 1.0074643665997427 seconds


In [7]:
# try a sparse solver

method = "sparse"
num_runs = 10
avg_time_sparse_solve = solver_demo.time_one_matrix_solve(K, R, method, num_runs)

print("average time sparse matrix solve:", avg_time_sparse_solve)


average time sparse matrix solve: 0.13919787499980885


In [ ]:
# try another sparse solver

method = "sparse_iterative"
num_runs = 10
avg_time_sparse_iterative_solve = solver_demo.time_one_matrix_solve(K, R, method, num_runs)

print("average time sparse iterative matrix solve:", avg_time_sparse_iterative_solve)


In [ ]:
# run the example to look at the results

nr_num_steps = 5
nr_print = True

displacements_all, nr_info_all = hyperelastic_solver(material_props, ele_type, coords.T, connect.T, fixed_nodes, dload_info, nr_print, nr_num_steps, nr_tol=1e-9, nr_maxit=30)

fname = "disp.gif"
viz.make_deformation_gif(displacements_all, coords, connect, ele_type, fname)
